In [16]:
#r "nuget: Microsoft.ML"

Installed Packages Microsoft.ML, 1.7.1

In [17]:
open System
open System.IO

In [18]:
open Microsoft.ML
open Microsoft.ML.Data

In [19]:
let _dataPath =
    Path.Combine(Environment.CurrentDirectory, "..", "Data", "yelp_labelled.txt")

In [20]:
type SentimentData =
    { [<LoadColumn 0>]
      SentimentText: string
      [<LoadColumn 1; ColumnName "Label">]
      Sentiment: bool }

[<CLIMutable>]
type SentimentPrediction =
    { [<ColumnName "PredictedLabel">]
      Prediction: bool
      Probability: single
      Score: single }

In [21]:
let mlcontext = MLContext()

In [22]:
let LoadData (mlContext: MLContext) =
    let dataView =
        mlContext.Data.LoadFromTextFile<SentimentData>(_dataPath, hasHeader = false)

    mlContext.Data.TrainTestSplit(dataView, testFraction = 0.2)

In [23]:
let splitDataView = LoadData(mlcontext)

In [24]:
let BuildAndTrainModel (mlContext: MLContext) splitTrainSet =
    mlContext
        .Transforms
        .Text
        .FeaturizeText(
            outputColumnName = "Features",
            inputColumnName = nameof Unchecked.defaultof<SentimentData>.SentimentText
        )
        .Append(
            mlContext.BinaryClassification.Trainers.SdcaLogisticRegression(
                labelColumnName = "Label",
                featureColumnName = "Features"
            )
        )
        .Fit(splitTrainSet)

In [25]:
printfn "=============== Create and Train the Model ==============="

let model = BuildAndTrainModel mlcontext splitDataView.TrainSet

printfn "=============== End of training ==============="

=============== Create and Train the Model ===============
=============== End of training ===============


In [26]:
let Evaluate (mlContext: MLContext) (model: ITransformer) (splitTestSet: IDataView) =
    let predictions = model.Transform(splitTestSet)
    mlContext.BinaryClassification.Evaluate(predictions, "Label")

In [27]:
printfn "=============== Evaluating Model accuracy with Test data==============="

let metrics = Evaluate mlcontext model splitDataView.TestSet

printfn "Model quality metrics evaluation"
printfn "--------------------------------"
printfn $"Accuracy: {metrics.Accuracy:P2}"
printfn $"Auc: {metrics.AreaUnderRocCurve:P2}"
printfn $"F1Score: {metrics.F1Score:P2}"
printfn "=============== End of model evaluation ==============="

=============== Evaluating Model accuracy with Test data===============
Model quality metrics evaluation
--------------------------------
Accuracy: 82.35 %
Auc: 90.05 %
F1Score: 83.25 %
=============== End of model evaluation ===============


In [28]:
let UseModelWithSingleItem (mlContext: MLContext) (model: ITransformer) =
    let predictionFunction =
        mlContext.Model.CreatePredictionEngine<SentimentData, SentimentPrediction>(model)
    { SentimentText = "This was a very bad steak"; Sentiment = false; }
    |> predictionFunction.Predict

In [29]:
printfn "=============== Prediction Test of model with a single sample and test dataset ==============="

let resultPrediction = UseModelWithSingleItem mlcontext model
printfn $"""Prediction: {if resultPrediction.Prediction then "Positive" else "Negative"} | Probability: {resultPrediction.Probability} """

printfn "=============== End of Predictions ==============="

=============== Prediction Test of model with a single sample and test dataset ===============
Prediction: Negative | Probability: 0.033318043 
=============== End of Predictions ===============
